# Lab 11 - Pruning Parcels

## Tasks

#### Task 1 - Proptype on 2004 parcel data

Build a query that will filter the 2004 parcel file

1. Read in the 2004 parce file
2. Create a column that contains True if the parcel is close to our 49 lakes (see lab 10) and flase otherwise. (Hint: create `lat_long` -> map to `code` -> check `isin` for the lakes we identified in lab 10).
3. Create a query that selects the correct columns, filters on the new column, then adds lake_names, lake_codes, and lake distance.  Finally filter by distance to the lake.

How much have we reduced the size of the parcel file?  Can you safely union all the files after these filters?

#### Task 2 - Read all 11 files and apply the process defined in the last task


#### Task 3 - Union all 11 files then write out the result.

In [1]:
from project_data_Quam import *
from project_cols_to_keep_and_drop import *

In [2]:
lakes_to_keep = lake_year_stats.DNR_ID_Site_Number.unique()

In [3]:
dtypes_dict = {'centroid_lat': str,
               'centroid_long': str}

In [4]:
import pandas as pd
example_parcel = pd.read_csv("./MinneMUDAC_raw_files/2004_metro_tax_parcels.txt", sep= "|", dtype=dtypes_dict)


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,3,4,5,6,7,10,12,17,20,22,24,25,26,27,28,29,30,31,32,33,36,37,38,44,45,48,49,50,52,53,55,56,57,58,59,60,61,62,63,64,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
example_parcel.head()

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,...,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long
0,0.0,8.03,NaN,NaN,N,NaN,NaN,NaN,SAINT FRANCIS,NaN,...,NaN,NaN,NaN,NaN,1980.0,2004,NaN,NaN,45.41332,-93.26739
1,0.0,0.93,NaN,NaN,N,NaN,24457,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1974.0,2004,55005,NaN,45.41354,-93.2701
2,0.0,8.75,NaN,NaN,N,NaN,24442,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1969.0,2004,55005,NaN,45.41318,-93.27344
3,0.0,11.17,NaN,NaN,N,NaN,410,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1989.0,2004,55005,NaN,45.41167,-93.27684
4,0.0,14.46,NaN,NaN,N,NaN,480,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1995.0,2004,55070,NaN,45.41169,-93.27849


In [6]:
rows_to_keep = (pd.Series(zip(example_parcel.centroid_lat, example_parcel.centroid_long))
                .map(lat_long_code_dict)
                .isin(lakes_to_keep)
               )

In [7]:
rows_to_keep.value_counts()

False    1795192
True      187183
dtype: int64

In [8]:
from dfply import make_symbolic

In [9]:
@make_symbolic
def col_zip(*cols):
    return pd.Series(zip(*cols))

In [10]:
make_lat_long = lambda chunk:(chunk
                              >> mutate(lat_long = pd.Series(zip(chunk.centroid_lat, chunk.centroid_long))))

In [11]:
import pandas as pd
read_parcel = lambda file: pd.read_csv(file, sep= "|", dtype=dtypes_dict)

In [15]:
simplify_parcel = lambda df:(df
                             >> select(cols_to_keep)
                             >> mutate(lat_long = pd.Series(zip(df.centroid_lat, df.centroid_long)))
                             >> mutate(rows_to_keep = (X.lat_long
                                                        .map(lat_long_code_dict)
                                                        .isin(lakes_to_keep)))
                             >> filter_by(X.rows_to_keep)
                             >> mutate(lake_name = X.lat_long.map(lat_long_name_dict),
                                       lake_code = X.lat_long.map(lat_long_code_dict),
                                       distance = X.lat_long.map(lat_long_dist))
                             >> filter_by(X.distance.astype(float) <= 1600))



In [38]:
ex = simplify_parcel(example_parcel)
ex.head()

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AG_PRESERV,BASEMENT,CITY,COOLING,DWELL_TYPE,EMV_BLDG,EMV_LAND,...,XUSE4_DESC,YEAR_BUILT,Year,centroid_lat,centroid_long,lat_long,rows_to_keep,lake_name,lake_code,distance
43581,0.0,3.30,NaN,N,NaN,LINO LAKES,NaN,NaN,0.0,1000.0,...,NaN,0.0,2004,45.19074,-93.08833,"(45.19074, -93.08833)",True,George Watch Lake,02000500-01,1593.945766
43619,0.0,2.70,NaN,N,NaN,LINO LAKES,NaN,NaN,134767.0,84245.0,...,NaN,1982.0,2004,45.19039,-93.09149,"(45.19039, -93.09149)",True,George Watch Lake,02000500-01,1565.733074
43620,0.0,0.78,NaN,N,NaN,LINO LAKES,NaN,NaN,116209.0,81310.0,...,NaN,1978.0,2004,45.19041,-93.09374,"(45.19041, -93.09374)",True,George Watch Lake,02000500-01,1599.159173
43626,0.0,0.72,NaN,N,NaN,LINO LAKES,NaN,NaN,151691.0,81310.0,...,NaN,1979.0,2004,45.19022,-93.09407,"(45.19022, -93.09407)",True,George Watch Lake,02000500-01,1584.864382
43627,0.0,1.18,NaN,N,NaN,LINO LAKES,NaN,NaN,137609.0,81310.0,...,NaN,1985.0,2004,45.19004,-93.09214,"(45.19004, -93.09214)",True,George Watch Lake,02000500-01,1534.365912


In [17]:
from glob import glob
files = glob('./MinneMUDAC_raw_files/****_metro_tax_parcels.txt')
FILE_NAME_RE = re.compile(r'^./MinneMUDAC_raw_files/([0-9]*)_metro_tax_parcels.txt$')
file_name = lambda p: FILE_NAME_RE.match(p).group(1) 
reduced_files = [file for file in files if file_name(file) not in ['2002', '2003', '2015']]

In [18]:
parcel_04_14_dfs = [simplify_parcel(read_parcel(file)) for file in reduced_files]




/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (2,3,4,5,6,7,10,12,17,19,20,21,22,24,25,26,27,28,29,30,31,32,33,34,37,38,39,45,46,49,50,51,54,56,57,58,59,60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (2,3,4,5,6,7,10,12,17,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,37,38,39,40,45,46,49,50,51,53,54,56,57,58,59,60,61,62,63,64,65,68) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (2,3,4,5,6,7,10,12,17,19,20,23,24,25,27,28,29,30,31,32,35,36,37,40,43,44,47,48,49,51,52,54,56,57,58,59,60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=Fal

In [21]:
parcel_04_14_dfs[0].columns

Index(['ACRES_DEED', 'ACRES_POLY', 'AGPRE_ENRD', 'AG_PRESERV', 'BASEMENT',
       'CITY', 'COOLING', 'DWELL_TYPE', 'EMV_BLDG', 'EMV_LAND', 'FIN_SQ_FT',
       'GARAGE', 'GARAGESQFT', 'GREEN_ACRE', 'HOMESTEAD', 'LANDMARK',
       'OWN_ADD_L1', 'OWN_ADD_L2', 'OWN_ADD_L3', 'PARC_CODE', 'PIN',
       'SALE_VALUE', 'SPEC_ASSES', 'TAX_CAPAC', 'TAX_EXEMPT', 'TOTAL_TAX',
       'USE1_DESC', 'USE2_DESC', 'USE3_DESC', 'USE4_DESC', 'WSHD_DIST',
       'XUSE1_DESC', 'XUSE2_DESC', 'XUSE3_DESC', 'XUSE4_DESC', 'YEAR_BUILT',
       'Year', 'centroid_lat', 'centroid_long', 'lat_long', 'rows_to_keep',
       'lake_name', 'lake_code', 'distance'],
      dtype='object')

In [22]:
cols = ['ACRES_DEED', 'ACRES_POLY', 'AGPRE_ENRD', 'AG_PRESERV', 'BASEMENT',
       'CITY', 'COOLING', 'DWELL_TYPE', 'EMV_BLDG', 'EMV_LAND', 'FIN_SQ_FT',
       'GARAGE', 'GARAGESQFT', 'GREEN_ACRE', 'HOMESTEAD', 'LANDMARK',
       'OWN_ADD_L1', 'OWN_ADD_L2', 'OWN_ADD_L3', 'PARC_CODE', 'PIN',
       'SALE_VALUE', 'SPEC_ASSES', 'TAX_CAPAC', 'TAX_EXEMPT', 'TOTAL_TAX',
       'USE1_DESC', 'USE2_DESC', 'USE3_DESC', 'USE4_DESC', 'WSHD_DIST',
       'XUSE1_DESC', 'XUSE2_DESC', 'XUSE3_DESC', 'XUSE4_DESC', 'YEAR_BUILT',
       'Year', 'centroid_lat', 'centroid_long', 'lat_long', 'rows_to_keep',
       'lake_name', 'lake_code', 'distance']

In [20]:
@dfpipe
def union_all(left_df, right_df, ignore_index=True):
    return pd.concat([left_df, right_df], ignore_index=ignore_index)

In [24]:
all_parcels_df = pd.DataFrame(columns=cols)
for df in parcel_04_14_dfs:
    all_parcels_df = all_parcels_df >> union_all(df)

In [82]:
all_parcels_df.TOTAL_TAX.unique()

array([  1694.,      0., 423734., ...,  14750.,  23504.,  26928.])

In [27]:
all_parcels_df.head()

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AG_PRESERV,BASEMENT,CITY,COOLING,DWELL_TYPE,EMV_BLDG,EMV_LAND,...,XUSE4_DESC,YEAR_BUILT,Year,centroid_lat,centroid_long,lat_long,rows_to_keep,lake_name,lake_code,distance
0,0.00,2.22,1899-12-30,N,NaN,LINO LAKES,NaN,RESIDENTIAL SINGLE FAMILY,18000.0,102800.0,...,NaN,0.0,2009,45.18883,-93.09643,"(45.18883, -93.09643)",True,George Watch Lake,02000500-01,1497.303125
1,4.87,4.65,1899-12-30,N,NaN,LINO LAKES,NaN,EXEMPT - COUNTY PUBLIC SERVICE,0.0,42000.0,...,NaN,0.0,2009,45.1825,-93.10329,"(45.1825, -93.10329)",True,George Watch Lake,02000500-01,1307.638376
2,0.00,15.95,1899-12-30,N,N,LINO LAKES,NaN,COMMERCIAL / GOLF COURSE,8645400.0,4130700.0,...,NaN,2002.0,2009,45.18437,-93.10511,"(45.18437, -93.10511)",True,George Watch Lake,02000500-01,1541.113584
3,0.00,3.74,1899-12-30,N,NaN,LINO LAKES,NaN,COMMERCIAL / GOLF COURSE,0.0,160000.0,...,NaN,0.0,2009,45.18447,-93.10292,"(45.18447, -93.10292)",True,George Watch Lake,02000500-01,1410.709951
4,39.98,35.38,1899-12-30,N,NaN,LINO LAKES,NaN,EXEMPT - COUNTY OTHER,0.0,220900.0,...,NaN,0.0,2009,45.16591,-93.0944,"(45.16591, -93.0944)",True,George Watch Lake,02000500-01,1239.856136


In [69]:
all_parcels_df.EMV_LAND.unique()

array([ 102800.,   42000., 4130700., ..., 2154000.,  751800., 1583900.])

In [40]:
from dfply import if_else

In [28]:
all_parcels_df.to_csv("./MinneMUDAC_raw_files/all_parcels.csv")

In [83]:
lake_year_parcel_info = (all_parcels_df
 >> select(X.ACRES_POLY, X.TOTAL_TAX, X.EMV_LAND, X.SALE_VALUE, X.AG_PRESERV,X.SPEC_ASSES, X.FIN_SQ_FT, X.lake_name, X.Year, X.lake_code)
 >> mutate(SALE_VALUE = if_else(X.SALE_VALUE == 0.0, float('NaN') , X.SALE_VALUE),
           SPEC_ASSES = if_else(X.SPEC_ASSES == 0.0, float('NaN') , X.SPEC_ASSES),
           FIN_SQ_FT = if_else(X.FIN_SQ_FT == 0.0, float('NaN') , X.FIN_SQ_FT),
           EMV_LAND = if_else(X.EMV_LAND == 0.0, float('NaN') , X.EMV_LAND),
           ACRES_POLY = if_else(X.ACRES_POLY == 0.0, float('NaN') , X.ACRES_POLY),
           TOTAL_TAXf = if_else(X.TOTAL_TAX == 0.0, float('NaN') , X.TOTAL_TAX))
 >> group_by(X.lake_name, X.Year, X.lake_code)
 >> summarise(mean_sale_value = mean(X.SALE_VALUE),
              overall_ag_preserv = n(X.AG_PRESERV == 'Y'),
              mean_fin_sqft = mean(X.FIN_SQ_FT),
              mean_land_value = mean(X.EMV_LAND),
              mean_acres_poly = mean(X.ACRES_POLY),
              mean_total_tax = mean(X.TOTAL_TAX)))


In [84]:
lake_year_parcel_info.head()

,lake_code,Year,lake_name,mean_sale_value,overall_ag_preserv,mean_fin_sqft,mean_land_value,mean_acres_poly,mean_total_tax
0,19002100-01,2004,Alimagnet Lake,171737.949367,2956,2201.410183,63836.812997,0.547422,2138.620095
1,19002100-01,2005,Alimagnet Lake,177982.662176,2962,2198.453501,69825.440110,0.547447,2264.375084
2,19002100-01,2006,Alimagnet Lake,183216.669039,2965,2198.877901,76715.742336,0.547888,2716.030691
3,19002100-01,2007,Alimagnet Lake,186377.522899,2968,2198.052500,80979.070568,0.547964,2683.084569
4,19002100-01,2008,Alimagnet Lake,188350.780952,2968,2196.843739,80766.127367,0.547964,2692.964286


In [85]:
len(lake_year_parcel_info)

528

In [89]:
lake_info = pd.read_csv("./MinneMUDAC_raw_files/lake_year_stats.csv")

In [90]:
len(lake_info)

539

In [92]:
combined_parcel_lake_info = lake_info >> left_join(lake_year_parcel_info, by=['lake_code', 'Year'])

In [93]:
combined_parcel_lake_info.to_csv("./MinneMUDAC_raw_files/combined_parcel_lake_info.csv")